In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import Levenshtein as Lv
import time
import re

import numpy as np
import json
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [21]:
fp = '../results/dblp-v10-grouped/2000-2004/AutoPhrase.txt'

In [32]:
yr = fp.split('/')[-2]

In [34]:
df = pd.read_csv(fp, sep='\t', header=None, names=['Phrase Quality', 'Phrase'])
df

,Phrase Quality,Phrase
0,0.971569,heat transfer
1,0.970683,belief propagation
2,0.969210,stock market
3,0.969203,congestion avoidance
4,0.969102,kalman filters
...,...,...
115039,0.009656,the increased use
115040,0.009610,both internal and
115041,0.009559,a picture of
115042,0.009355,the difficulty to


In [35]:
df['Year'] = [yr] * len(df)

In [36]:
df

,Phrase Quality,Phrase,Year
0,0.971569,heat transfer,2000-2004
1,0.970683,belief propagation,2000-2004
2,0.969210,stock market,2000-2004
3,0.969203,congestion avoidance,2000-2004
4,0.969102,kalman filters,2000-2004
...,...,...,...
115039,0.009656,the increased use,2000-2004
115040,0.009610,both internal and,2000-2004
115041,0.009559,a picture of,2000-2004
115042,0.009355,the difficulty to,2000-2004


In [38]:
df = df.dropna()
df

,Phrase Quality,Phrase,Year
0,0.971569,heat transfer,2000-2004
1,0.970683,belief propagation,2000-2004
2,0.969210,stock market,2000-2004
3,0.969203,congestion avoidance,2000-2004
4,0.969102,kalman filters,2000-2004
...,...,...,...
115039,0.009656,the increased use,2000-2004
115040,0.009610,both internal and,2000-2004
115041,0.009559,a picture of,2000-2004
115042,0.009355,the difficulty to,2000-2004


In [46]:
df['Num Words'] = df['Phrase'].map(str.split).map(len)
df

C:\Users\itsno\AppData\Local\Temp/ipykernel_25164/60564808.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Num Words'] = df['Phrase'].map(str.split).map(len)


,Phrase Quality,Phrase,Year,Num Words
0,0.971569,heat transfer,2000-2004,2
1,0.970683,belief propagation,2000-2004,2
2,0.969210,stock market,2000-2004,2
3,0.969203,congestion avoidance,2000-2004,2
4,0.969102,kalman filters,2000-2004,2
...,...,...,...,...
115039,0.009656,the increased use,2000-2004,3
115040,0.009610,both internal and,2000-2004,3
115041,0.009559,a picture of,2000-2004,3
115042,0.009355,the difficulty to,2000-2004,3


In [55]:
fun = lambda x: True if (x['Num Words'] > 1 and x['Phrase Quality'] >= 0.5) or (x['Num Words'] == 1 and x['Phrase Quality'] >= 0.8) else False

In [58]:
valid = df.apply(lambda x: fun(x), axis=1)

In [65]:
df.apply(fun, axis=1)

0          True
1          True
2          True
3          True
4          True
          ...  
115039    False
115040    False
115041    False
115042    False
115043    False
Length: 115043, dtype: bool

In [59]:
df[valid]

,Phrase Quality,Phrase,Year,Num Words
0,0.971569,heat transfer,2000-2004,2
1,0.970683,belief propagation,2000-2004,2
2,0.969210,stock market,2000-2004,2
3,0.969203,congestion avoidance,2000-2004,2
4,0.969102,kalman filters,2000-2004,2
...,...,...,...,...
54273,0.500097,verification algorithms,2000-2004,2
54274,0.500050,synchronization constraints,2000-2004,2
54275,0.500048,effective integration,2000-2004,2
54276,0.500038,produce high quality,2000-2004,3


In [61]:
df['Num Words'].value_counts()

2    54078
3    29479
1    23648
4     6804
5      870
6      164
Name: Num Words, dtype: int64

In [66]:
df

,Phrase Quality,Phrase,Year,Num Words
0,0.971569,heat transfer,2000-2004,2
1,0.970683,belief propagation,2000-2004,2
2,0.969210,stock market,2000-2004,2
3,0.969203,congestion avoidance,2000-2004,2
4,0.969102,kalman filters,2000-2004,2
...,...,...,...,...
115039,0.009656,the increased use,2000-2004,3
115040,0.009610,both internal and,2000-2004,3
115041,0.009559,a picture of,2000-2004,3
115042,0.009355,the difficulty to,2000-2004,3


In [67]:
df2 = df.copy()
df2

,Phrase Quality,Phrase,Year,Num Words
0,0.971569,heat transfer,2000-2004,2
1,0.970683,belief propagation,2000-2004,2
2,0.969210,stock market,2000-2004,2
3,0.969203,congestion avoidance,2000-2004,2
4,0.969102,kalman filters,2000-2004,2
...,...,...,...,...
115039,0.009656,the increased use,2000-2004,3
115040,0.009610,both internal and,2000-2004,3
115041,0.009559,a picture of,2000-2004,3
115042,0.009355,the difficulty to,2000-2004,3


In [71]:
df = df.append(df2, ignore_index=True)

In [72]:
df

,Phrase Quality,Phrase,Year,Num Words
0,0.971569,heat transfer,2000-2004,2
1,0.970683,belief propagation,2000-2004,2
2,0.969210,stock market,2000-2004,2
3,0.969203,congestion avoidance,2000-2004,2
4,0.969102,kalman filters,2000-2004,2
...,...,...,...,...
230081,0.009656,the increased use,2000-2004,3
230082,0.009610,both internal and,2000-2004,3
230083,0.009559,a picture of,2000-2004,3
230084,0.009355,the difficulty to,2000-2004,3


## Looking at segmentation + AutoPhrase results for grouped years

In [3]:
grouped_fp = '../results/dblp-v10-grouped'
seg_fp = grouped_fp + '/segmentation/2000-2004_segmented.csv'
auto_fp = grouped_fp + '/dblp-v10-grouped-phrases.csv'

In [5]:
phrases = pd.read_csv(auto_fp, index_col=0)
#phrases = phrases[phrases['Year']=='2000-2004']
phrases

,Phrase Quality,Phrase,Year
0,0.991500,operations research,1950-1959
1,0.650500,operations research society of america,1950-1959
2,0.573500,high speed,1950-1959
3,0.525500,operations research society,1950-1959
4,0.991500,operations research,1950-1959
...,...,...,...
258449,0.855142,natural language understanding,2015-2017
258450,0.855141,ganglion cells,2015-2017
258451,0.855137,underwater acoustic networks,2015-2017
258452,0.855132,small base stations sbss,2015-2017


In [7]:
phrases['Year'].unique()

array(['1950-1959', '1960-1964', '1965-1969', '1970-1974', '1975-1979',
       '1980-1984', '1985-1989', '1990-1994', '1995-1999', '2000-2004',
       '2005-2009', '2010-2014', '2015-2017'], dtype=object)

In [8]:
seg = pd.read_csv(seg_fp, index_col=0)
seg = seg.dropna() # Need to drop null values to prevent issues with string processing
seg

,Phrases,Year
0,"hamiltonian,wave,periodic boundary conditions,...",2000-2004
1,"embedded systems,memory,paper,architecture,dat...",2000-2004
2,"language modeling,information,single,word orde...",2000-2004
3,"mirror,software,perl,user-controlled,mirror,we...",2000-2004
4,"covariance,regression,reproducing kernel hilbe...",2000-2004
...,...,...
309699,"prototype,walking robot,ship,legged robot,acti...",2000-2004
309700,"paper,parallel program,in grid,order,user inte...",2000-2004
309701,"search-based,cbr,ca,ca,search process,cbr,ca,g...",2000-2004
309702,"dram,production,tests,single,single,memory,tes...",2000-2004


In [10]:
seg.loc[0]['Phrases']

'hamiltonian,wave,periodic boundary conditions,paper,nonlinear schrodinger,symplectic,symplectic,data,multi-phase,spectral methods'

In [13]:
def find_qualities(x):
    x = x['Phrases'].split(',')
    out_phrases = []
    out_quality = []
    for phrase in x:
        # Phrase will not show up in the df if the quality is too low
        # We only kept multi >= 0.5 and single >= 0.8
        # NOTE: Potential issue with phrase having dashes in seg when they don't in phrases
        #       (i.e. user-controlled vs. user controlled)
        match = phrases[phrases['Phrase'] == phrase]
        if len(match) == 0:
            continue
        else:
            out_phrases.append(phrase)
            out_quality.append(match['Phrase Quality'].values[0])
    return out_phrases, out_quality

In [14]:
test = seg.copy()
test = seg.loc[1:10]

In [15]:
test['Phrases'], test['Phrase Qualities'] = zip(*test.apply(find_qualities, axis=1))

C:\Users\itsno\AppData\Local\Temp/ipykernel_25164/2623782444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Phrases'], test['Phrase Qualities'] = zip(*test.apply(find_qualities, axis=1))


In [20]:
test

,Phrases,Year,Phrase Qualities
1,"[embedded systems, control flow, program trans...",2000-2004,"[0.751535651, 0.9272791598, 0.8154624019, 0.68..."
2,"[language modeling, word order, language model...",2000-2004,"[0.7877440842, 0.8562616285, 0.7877440842, 0.8..."
3,"[mirror, perl, mirror, web cache, http, mirror...",2000-2004,"[0.8339330792, 0.824974653, 0.8339330792, 0.72..."
4,"[covariance, regression, reproducing kernel hi...",2000-2004,"[0.8073973255, 0.8220048591, 0.9075834826, 0.6..."
5,"[modular exponentiation, cornerstone, rsa, mod...",2000-2004,"[0.8909553133, 0.8050896539, 0.807428333, 0.74..."
6,"[question answering, ir, semantic, semantic, n...",2000-2004,"[0.9793318108, 0.8065003229, 0.8092162159, 0.8..."
7,"[todays web, web services, markup, web service...",2000-2004,"[0.5229487085, 0.8034459036, 0.8582076273, 0.8..."
8,"[mediaeval, logic, aristotelian, negation, neg...",2000-2004,"[0.8598218899, 0.8114334306, 0.8674139965, 0.8..."
9,"[intelligent systems, previous attempts, searc...",2000-2004,"[0.7989019338, 0.5292607943, 0.6592930534, 0.8..."
10,"[noise robust, speech recognition, aurora, pre...",2000-2004,"[0.6364751042, 0.9559212685, 0.8590855236, 0.7..."


In [18]:
test.loc[1]['Phrases']

['embedded systems',
 'control flow',
 'program transformations',
 'embedded applications',
 'library',
 'code size']

In [19]:
test.loc[1]['Phrase Qualities']

[0.751535651,
 0.9272791598,
 0.8154624019,
 0.6875766847,
 0.8212738409,
 0.7391568024]

In [ ]:
seg

In [145]:
phrases[phrases['Phrase'] == 'user controlled']

,Phrase Quality,Phrase,Year
45759,0.716653,user controlled,2000-2004


In [113]:
re.sub(r'[^A-Za-z0-9- ]+', '', 'user-controlled')

'user-controlled'

In [116]:
re.sub(r'[-]+', ' ', 'user-controlled')

'user controlled'

# Analysis of AutoPhrase results

In [3]:
fp_uniquebyyear = '../results/dblp-v10-phrases-uniquebyyear.csv'
fp_unique = '../results/dblp-v10-phrases-unique.csv'

In [4]:
# Start from year 1968 - years before had too little training data
# We only kept multi-word phrases above 0.6 and single-word above 0.8

In [63]:
# Contains the phrases unique overall (no duplicates)
un_all = pd.read_csv(fp_unique, index_col=0)
un_all = un_all[un_all['Year'] >= 1968]
un_all

,Phrase Quality,Phrase,Year
9,0.890167,time sharing,1968
10,0.610000,real time,1970
11,0.964167,pattern recognition,1972
12,0.866167,data base,1972
13,0.850167,programming languages,1972
...,...,...,...
39336,0.603183,great attention,2017
39337,0.602875,limited training,2017
39338,0.602173,public datasets,2017
39339,0.602145,existing works,2017


In [6]:
# Contains the phrases unique by year (there can be duplicates across years)
uby = pd.read_csv(fp_uniquebyyear, index_col=0)
uby = uby[uby['Year'] >= 1968]
uby = uby.dropna()
uby

,Phrase Quality,Phrase,Year
20,0.946833,context free,1968
21,0.890167,time sharing,1968
22,0.909000,context free,1969
23,0.896000,time sharing,1969
24,0.993000,context free,1970
...,...,...,...
238718,0.600501,practical implementation,2017
238719,0.600363,next generation,2017
238720,0.600185,deep convolutional neural,2017
238721,0.600087,network nodes,2017


In [7]:
# Only keeps the phrases that show up multiple times across years (so we can look for trends)
uby_dups = uby.copy()
uby_dups = uby_dups[uby_dups['Phrase'].duplicated(keep=False)]
uby_dups

,Phrase Quality,Phrase,Year
20,0.946833,context free,1968
21,0.890167,time sharing,1968
22,0.909000,context free,1969
23,0.896000,time sharing,1969
24,0.993000,context free,1970
...,...,...,...
238718,0.600501,practical implementation,2017
238719,0.600363,next generation,2017
238720,0.600185,deep convolutional neural,2017
238721,0.600087,network nodes,2017


In [68]:
uby_dups['Phrase'].value_counts()[:10]

context free             46
high level               42
programming language     42
natural language         41
pattern recognition      41
programming languages    41
data structures          41
sufficient conditions    39
data structure           39
problem solving          39
Name: Phrase, dtype: int64

In [67]:
uby_dups[uby_dups['Phrase'] == 'image processing']['Year'].values

array([1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017], dtype=int64)

# Phrase matching/similarity for input papers (title + abstract)

In [11]:
# Given an input paper (title + abstract), extract the phrases within it and return the
# similar phrases found in the AutoPhrase results.
# Can use Levenshtein distance to find similar strings, or just use direct phrase matching

In [12]:
test_fp = '../data/arxiv/csv/2016.csv'

In [13]:
test_data = pd.read_csv(test_fp)
test_data.head()

,Title,Abstract
0,Sequential Short-Text Classification with Recu...,Recent approaches based on artificial neural n...
1,Multiresolution Recurrent Neural Networks An A...,We introduce the multiresolution recurrent neu...
2,Document Image Coding and Clustering for Scrip...,The paper introduces a new method for discrimi...
3,Tutorial on Answering Questions about Images w...,Together with the development of more accurate...
4,Building Machines That Learn and Think Like Pe...,Recent progress in artificial intelligence AI ...


In [14]:
test_data['Title'][0] + ' ' + test_data['Abstract'][0]

'Sequential Short-Text Classification with Recurrent and Convolutional  Neural Networks Recent approaches based on artificial neural networks ANNs have shownpromising results for short-text classification However many short textsoccur in sequences eg sentences in a document or utterances in a dialogand most existing ANN-based systems do not leverage the preceding short textswhen classifying a subsequent one In this work we present a model based onrecurrent neural networks and convolutional neural networks that incorporatesthe preceding short texts Our model achieves state-of-the-art results on threedifferent datasets for dialog act prediction'

In [15]:
uby

,Phrase Quality,Phrase,Year
20,0.946833,context free,1968
21,0.890167,time sharing,1968
22,0.909000,context free,1969
23,0.896000,time sharing,1969
24,0.993000,context free,1970
...,...,...,...
238718,0.600501,practical implementation,2017
238719,0.600363,next generation,2017
238720,0.600185,deep convolutional neural,2017
238721,0.600087,network nodes,2017


In [16]:
input_phrase = 'convolutional neural networks'

In [17]:
valid_ix = uby.apply(lambda x: x['Phrase'][0] == 'c' if isinstance(x['Phrase'], str) else False, axis=1)
valid_ix

20         True
21        False
22         True
23        False
24         True
          ...  
238718    False
238719    False
238720    False
238721    False
238722    False
Length: 238701, dtype: bool

In [18]:
unique_phrases = uby[valid_ix]['Phrase'].unique()
unique_phrases

array(['context free', 'computer science', 'computer graphics', ...,
       'complete characterization', 'co occurrence matrix',
       'computational approach'], dtype=object)

In [19]:
candidate = ''
dist = float('inf')
for phrase in unique_phrases:
    diff = Lv.distance(input_phrase, phrase)
    if diff < dist:
        candidate = phrase
        dist = diff

In [20]:
candidate

'convolutional neural networks'

In [21]:
dist

0

In [22]:
uby[uby['Phrase'] == 'convolutional neural networks']

,Phrase Quality,Phrase,Year
136900,0.865809,convolutional neural networks,2012
152925,0.915629,convolutional neural networks,2013
172010,0.937014,convolutional neural networks,2014
190695,0.931728,convolutional neural networks,2015
212473,0.917273,convolutional neural networks,2016
233574,0.904261,convolutional neural networks,2017


In [23]:
# Same approach, but keeping track of all candidates this time

In [24]:
candidates = []
for phrase in unique_phrases:
    dist = Lv.distance(input_phrase, phrase)
    candidates.append((dist, phrase))

In [25]:
candidates.sort()

In [26]:
candidates[:10]

[(0, 'convolutional neural networks'),
 (1, 'convolutional neural network'),
 (3, 'convolution neural network'),
 (4, 'convolutional neural network cnn'),
 (4, 'convolutional neural networks cnn'),
 (5, 'convolutional neural networks cnns'),
 (7, 'convolutional networks'),
 (8, 'convolutional network'),
 (9, 'cellular neural networks'),
 (9, 'chaotic neural networks')]

In [27]:
# Using df.apply so we can look at all phrases, not just phrases that start with the same letter

In [28]:
uby_test = uby.copy()
uby_test['Dist'] = uby_test.apply(lambda x: Lv.distance(input_phrase, x['Phrase']) if isinstance(x['Phrase'], str) else float('inf'), axis=1)
uby_test

,Phrase Quality,Phrase,Year,Dist
20,0.946833,context free,1968,22
21,0.890167,time sharing,1968,24
22,0.909000,context free,1969,22
23,0.896000,time sharing,1969,24
24,0.993000,context free,1970,22
...,...,...,...,...
238718,0.600501,practical implementation,2017,22
238719,0.600363,next generation,2017,21
238720,0.600185,deep convolutional neural,2017,14
238721,0.600087,network nodes,2017,21


In [29]:
uby_test.sort_values('Dist')[:10]

,Phrase Quality,Phrase,Year,Dist
212473,0.917273,convolutional neural networks,2016,0
233574,0.904261,convolutional neural networks,2017,0
152925,0.915629,convolutional neural networks,2013,0
190695,0.931728,convolutional neural networks,2015,0
136900,0.865809,convolutional neural networks,2012,0
172010,0.937014,convolutional neural networks,2014,0
190443,0.936880,convolutional neural network,2015,1
174069,0.898522,convolutional neural network,2014,1
212157,0.922546,convolutional neural network,2016,1
153942,0.900172,convolutional neural network,2013,1


In [30]:
# Testing on the unique overall df

In [31]:
un_all_test = un_all.copy()
un_all_test['Dist'] = un_all_test.apply(lambda x: Lv.distance(input_phrase, x['Phrase']) if isinstance(x['Phrase'], str) else float('inf'), axis=1)
un_all_test.head()

,Phrase Quality,Phrase,Year,Dist
9,0.890167,time sharing,1968,24.0
10,0.610000,real time,1970,24.0
11,0.964167,pattern recognition,1972,22.0
12,0.866167,data base,1972,24.0
13,0.850167,programming languages,1972,22.0


In [32]:
un_all_test.sort_values('Dist')[:20]

,Phrase Quality,Phrase,Year,Dist
27755,0.865809,convolutional neural networks,2012,0.0
30221,0.900172,convolutional neural network,2013,1.0
37333,0.839879,convolution neural network,2016,3.0
34364,0.918423,convolutional neural networks cnn,2015,4.0
32655,0.915458,convolutional neural network cnn,2014,4.0
32739,0.889687,deep convolutional neural networks,2014,5.0
32650,0.918479,convolutional neural networks cnns,2014,5.0
32720,0.894009,deep convolutional neural network,2014,6.0
33724,0.726266,convolutional networks,2014,7.0
34288,0.608522,convolutional network,2014,8.0


In [33]:
# Consolidating results
# Can start with the most common phrases and change phrases that are close enough (distance <= 5?)
counts = uby.groupby('Phrase').size()

In [34]:
# num_years tells us how many years the phrase has shown up in
uby['num_years'] = uby.apply(lambda x: counts[x['Phrase']], axis=1)

In [35]:
uby.head()

,Phrase Quality,Phrase,Year,num_years
20,0.946833,context free,1968,46
21,0.890167,time sharing,1968,2
22,0.909000,context free,1969,46
23,0.896000,time sharing,1969,2
24,0.993000,context free,1970,46


# Model generation

In [36]:
# x: Phrase, Phrase Quality, num_years
# Phrase needs to be one hot encoded
# y: Only the year (may need to use the unique overall dataframe?)

# For phrase quality - standard scaler by year?
# For num_years - normalize overall

In [37]:
from sklearn.preprocessing import StandardScaler
std_pipe = Pipeline([('scale', StandardScaler())])
ohe_pipe = Pipeline([('one-hot', OneHotEncoder(handle_unknown='ignore'))])
ct = ColumnTransformer(transformers=[('ohe', ohe_pipe, ['Phrase']),
                                     ('scale', std_pipe, ['num_years']),
                                    ('keep', 'passthrough', ['Phrase Quality'])])
pl = Pipeline([('transform', ct), ('classifier', DecisionTreeClassifier())])

In [38]:
# Set random seed
X_train, X_test, y_train, y_test = train_test_split(uby[['Phrase', 'num_years', 'Phrase Quality']],
                                                    uby['Year'],
                                                    random_state=1)

In [39]:
_ = pl.fit(X_train, y_train)

In [40]:
# Mean accuracy - 8% accurate
pl.score(X_test, y_test)

0.08087003150345197

In [41]:
X_test['Predicted Year'] = pl.predict(X_test)
X_test['Year'] = y_test
X_test['Abs Year Diff'] = abs(X_test['Year'] - X_test['Predicted Year'])

In [42]:
X_test

,Phrase,num_years,Phrase Quality,Predicted Year,Year,Abs Year Diff
198146,band selection,11,0.831558,2011,2015,4
152976,mixed criticality,7,0.914890,2015,2013,2
200739,rna seq data,7,0.807498,2012,2015,3
238274,final result,1,0.642976,2016,2017,1
142017,presidential,4,0.806453,2015,2012,3
...,...,...,...,...,...,...
64293,treatment planning,12,0.792733,2010,2007,3
237679,default mode,10,0.696536,2009,2017,8
198475,user behaviour,12,0.828791,2012,2015,3
5578,temporal reasoning,17,0.905742,2009,1996,13


In [43]:
X_test['Abs Year Diff'].mean()

4.816358335009049

In [44]:
# THINGS TO TEST:
# Using the unique by year dataframe, then only keeping one instance of duplicate phrases
# but replace the year with the average (or median) of the years
# Normalizing the numeric features

# Try using phrasal segmentation model to run on a single paper title + abstract
# Or a single paper full paper text

# Try grouping by papers by a range of years (maybe 5)

In [45]:
X_train[X_train['Phrase']=='convolutional neural networks']

,Phrase,num_years,Phrase Quality
212473,convolutional neural networks,6,0.917273
152925,convolutional neural networks,6,0.915629
190695,convolutional neural networks,6,0.931728
136900,convolutional neural networks,6,0.865809


In [46]:
test = pd.DataFrame([[1.0, 'convolutional neural networks', 2005, 6]], columns=['Phrase Quality', 'Phrase', 'Year', 'num_years'])
test

,Phrase Quality,Phrase,Year,num_years
0,1.0,convolutional neural networks,2005,6


In [47]:
pl.predict(test[['Phrase', 'num_years', 'Phrase Quality']])

array([1991], dtype=int64)

In [48]:
uby[uby['Phrase'] == 'convolutional neural networks']

,Phrase Quality,Phrase,Year,num_years
136900,0.865809,convolutional neural networks,2012,6
152925,0.915629,convolutional neural networks,2013,6
172010,0.937014,convolutional neural networks,2014,6
190695,0.931728,convolutional neural networks,2015,6
212473,0.917273,convolutional neural networks,2016,6
233574,0.904261,convolutional neural networks,2017,6


In [49]:
test = pd.DataFrame([[1.0, 'artificial intelligence', 2005, 6]], columns=['Phrase Quality', 'Phrase', 'Year', 'num_years'])
test

,Phrase Quality,Phrase,Year,num_years
0,1.0,artificial intelligence,2005,6


In [50]:
pl.predict(test[['Phrase', 'num_years', 'Phrase Quality']])

array([1991], dtype=int64)

# Baseline model testing with grouped phrases

In [51]:
df = pd.read_csv('../results/dblp-v10-grouped/dblp-v10-grouped-phrases.csv', index_col=0)
df = df[4:]
df = df.dropna()

In [52]:
# num_years is slightly different since we grouped years now
counts = df.groupby('Phrase').size()
df['num_years'] = df.apply(lambda x: counts[x['Phrase']], axis=1)

In [53]:
df.head()

,Phrase Quality,Phrase,Year,num_years
4,0.9915,operations research,1950-1959,9
5,0.6505,operations research society of america,1950-1959,1
6,0.5735,high speed,1950-1959,13
7,0.5255,operations research society,1950-1959,1
8,0.9810,tunnel diode,1960-1964,1


In [54]:
std_pipe = Pipeline([('scale', StandardScaler())])
ohe_pipe = Pipeline([('one-hot', OneHotEncoder(handle_unknown='ignore'))])
ct = ColumnTransformer(transformers=[('ohe', ohe_pipe, ['Phrase']),
                                     ('scale', std_pipe, ['num_years']),
                                    ('keep', 'passthrough', ['Phrase Quality'])])
pl = Pipeline([('transform', ct), ('classifier', DecisionTreeClassifier())])

In [55]:
X_train, X_test, y_train, y_test = train_test_split(df[['Phrase', 'num_years', 'Phrase Quality']],
                                                    df['Year'],
                                                    random_state=1)

In [56]:
_ = pl.fit(X_train, y_train)

In [57]:
# 42.5% accuracy
pl.score(X_test, y_test)

0.4248282052869436

In [58]:
X_test['Predicted Year'] = pl.predict(X_test)
X_test['Year'] = y_test

In [59]:
X_test

,Phrase,num_years,Phrase Quality,Predicted Year,Year
222676,difference map,1,0.605864,2010-2014,2010-2014
145496,solid state drives ssds,2,0.907548,2010-2014,2010-2014
90177,transition fault coverage,2,0.811149,2005-2009,2005-2009
47354,self localization,4,0.693821,2000-2004,2000-2004
142852,npc inverter,1,0.918717,2010-2014,2010-2014
...,...,...,...,...,...
10352,concluding remarks,4,0.918754,2010-2014,1995-1999
118342,power utility,1,0.623109,2010-2014,2005-2009
243426,young adults,4,0.944511,2010-2014,2015-2017
251990,parabolic partial differential equations,4,0.882640,2010-2014,2015-2017


In [60]:
# How does the accuracy compare to just guessing the most common?
X_test['Year'].value_counts()

2010-2014    26721
2005-2009    19052
2000-2004     8605
2015-2017     3932
1995-1999     3901
1990-1994     1583
1985-1989      524
1980-1984      168
1975-1979       72
1970-1974       37
1965-1969       14
1950-1959        2
1960-1964        1
Name: Year, dtype: int64

In [61]:
(y_test == '2010-2014').mean()

0.41356094843063207